In [ ]:
!pip install numpy --upgrade --force-reinstall
!pip install torch --upgrade
!pip install transformers --upgrade
!pip install diffusers --upgrade
!pip install wikipedia requests librosa pathlib python-pptx TTS pillow google-generativeai gTTS

import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 843.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
import torch
import wikipedia
import requests
import os
import textwrap
import io
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from diffusers import StableDiffusionPipeline, DiffusionPipeline
from diffusers.utils import export_to_video
from TTS.api import TTS
import numpy as np
import google.generativeai as genai
from gtts import gTTS
from pathlib import Path

import re

def classify_task_nlp(user_input):
    user_input = user_input.lower()
    if re.search(r"\b(text|write|generate.*text|story|paragraph|explanation)\b", user_input):
        return "text"
    elif re.search(r"\b(image|picture|generate.*image|draw|visual|graphic)\b", user_input):
        return "image"
    elif re.search(r"\b(audio|sound|speech|generate.*audio|voice)\b", user_input):
        return "speech"
    elif re.search(r"\b(ppt|presentation|slides)\b", user_input):
        return "ppt"
    elif re.search(r"\b(video|animation|clip|generate.*video|movie)\b", user_input):
        return "video"
    return None

# Install required libraries
# (Commented out for production; use requirements.txt)
# os.system("pip install torch wikipedia requests librosa numpy pathlib python-pptx diffusers transformers TTS pillow google-generativeai gTTS")

DEFAULT_IMAGE = "default_placeholder.jpg"

# Load Stable Diffusion model
print("Loading Stable Diffusion model...")
model_id = "stabilityai/stable-diffusion-2"
stable_diffusion = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")

# Load TTS model
print("Loading TTS model...")
tts_model = TTS("tts_models/en/ljspeech/tacotron2-DDC").to("cuda")

# Load Video Diffusion pipeline
print("Loading Video Diffusion model...")
video_pipe = DiffusionPipeline.from_pretrained(
    "cerspense/zeroscope_v2_576w",
    torch_dtype=torch.float16
).to("cuda")

# Configure Gemini model
print("Loading Gemini AI...")
genai.configure(api_key="YOUR_API_KEY_HERE")  # Replace with your actual API key
gemini_model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Optional CUDA tweaks
torch.backends.cuda.matmul.allow_tf32 = True

def generate_text(prompt):
    try:
        response = gemini_model.generate_content(prompt)
        text_response = response.text
        print("📝 Generated text:\n", text_response)

        # Save and notify audio
        tts = gTTS(text_response)
        tts.save("generated_text.mp3")
        print("🔊 Response also saved as generated_text.mp3 (play manually if needed)")
    except Exception as e:
        print(f"⚠️ Gemini Error: {str(e)}")

def generate_image(prompt):
    try:
        image = stable_diffusion(prompt, num_inference_steps=35, guidance_scale=9).images[0]
        image.save("generated_image.png")
        print("🖼️ Image generated and saved as generated_image.png")
    except Exception as e:
        print(f"Failed to generate image: {e}")

def generate_audio(text):
    try:
        tts_model.tts_to_file(text=text, file_path="generated_sound.wav")
        print("🔈 Audio generated and saved as generated_sound.wav")
    except Exception as e:
        print(f"Failed to generate audio: {e}")

def generate_video(prompt):
    try:
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

        print("🎬 Generating video frames...")
        video_output = video_pipe(
            prompt=prompt,
            width=384,
            height=256,
            num_frames=24,
            num_inference_steps=25,
        ).frames

        frames = video_output[0]
        print(f"Frames shape: {frames.shape}, dtype: {frames.dtype}")

        processed_frames = []
        for frame in frames:
            frame = np.clip(frame * 255, 0, 255).astype(np.uint8)
            processed_frames.append(frame)

        export_to_video(processed_frames, "output.mp4", fps=24)
        print("🎥 Video saved as output.mp4")
    except Exception as e:
        print(f"Failed to generate video: {e}")

def get_best_wikipedia_page(topic):
    try:
        search_results = wikipedia.search(topic)
        return search_results[0] if search_results else topic
    except Exception as e:
        print(f"Error searching Wikipedia: {e}")
        return topic

def fetch_image(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic.replace(' ', '_')}"
    try:
        response = requests.get(url).json()
        if "thumbnail" in response and "source" in response["thumbnail"]:
            return response["thumbnail"]["source"]
    except Exception as e:
        print(f"Error fetching image: {e}")
    return None

def download_image(image_url):
    if not image_url:
        return DEFAULT_IMAGE
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        img = Image.open(io.BytesIO(response.content))
        img_format = img.format.lower()
        if img_format not in ["jpeg", "png"]:
            print(f"Unsupported image format: {img_format}")
            return DEFAULT_IMAGE
        img_path = f"topic_image.{img_format}"
        img.save(img_path)
        return img_path
    except Exception as e:
        print(f"Error downloading image: {e}")
        return DEFAULT_IMAGE

def generate_slide_content(topic):
    topic = get_best_wikipedia_page(topic)
    try:
        summary = wikipedia.summary(topic, sentences=10)
        wrapped_text = textwrap.wrap(summary, width=200)
        return wrapped_text
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error: {e.options[:5]}")
    except wikipedia.exceptions.PageError:
        print("Page not found.")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None

def create_ppt(topic, content):
    prs = Presentation()
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = topic
    subtitle.text = "A Comprehensive Overview"

    image_url = fetch_image(topic)
    img_path = download_image(image_url)

    if img_path:
        slide.shapes.add_picture(img_path, Inches(6), Inches(1), Inches(3), Inches(2))

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Introduction"
    content_box.text = f"This presentation provides an overview of {topic}."

    subtopics = ["Background", "Main Concepts", "Applications", "Advantages", "Challenges", "Future Scope"]

    for i in range(len(subtopics)):
        slide_layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        content_box = slide.placeholders[1]
        title.text = subtopics[i]
        content_box.text = content[i] if i < len(content) else "More details available online."

    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    content_box = slide.placeholders[1]
    title.text = "Conclusion"
    content_box.text = f"Thank you for exploring {topic}."

    ppt_filename = f"{topic.replace(' ', '_')}.pptx"
    prs.save(ppt_filename)
    print(f"📊 Presentation saved as {ppt_filename}")

    if img_path and img_path != DEFAULT_IMAGE:
        os.remove(img_path)

import ipywidgets as widgets
from IPython.display import display, Audio, Image as IPImage, Video as IPVideo, FileLink, clear_output

# Heading
header = widgets.HTML(value="<h1 style='text-align:center;'>Generative AI with Blockchain</h1>")
display(header)

# Input and Dropdown
prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your prompt or topic here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)
generate_type = widgets.Dropdown(
    options=['text', 'image', 'speech', 'video', 'ppt'],
    value='text',
    description='Generate:',
)
run_button = widgets.Button(description="Generate", button_style='success')
output_box = widgets.Output()

def run_generation(_):
    output_box.clear_output()
    prompt = prompt_input.value
    kind = generate_type.value

    with output_box:
        if kind == 'text':
            generate_text(prompt)
            display(Audio("generated_text.mp3"))
            display(FileLink("generated_text.mp3", result_html_prefix="Download: "))

        elif kind == 'image':
            generate_image(prompt)
            display(IPImage("generated_image.png"))
            display(FileLink("generated_image.png", result_html_prefix="Download: "))

        elif kind == 'speech':
            generate_audio(prompt)
            display(Audio("generated_sound.wav"))
            display(FileLink("generated_sound.wav", result_html_prefix="Download: "))

        elif kind == 'video':
            generate_video(prompt)
            display(IPVideo("output.mp4", embed=True))
            display(FileLink("output.mp4", result_html_prefix="Download: "))

        elif kind == 'ppt':
            content = generate_slide_content(prompt)
            if content:
                create_ppt(prompt, content)
                ppt_name = f"{prompt.replace(' ', '_')}.pptx"
                display(FileLink(ppt_name, result_html_prefix="Download: "))
            else:
                print("Could not generate PPT content.")

run_button.on_click(run_generation)

# Layout
ui = widgets.VBox([
    prompt_input,
    generate_type,
    run_button,
    output_box
])

display(ui)


Loading Stable Diffusion model...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading TTS model...
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Loading Gemini AI...


HTML(value="<h1 style='text-align:center;'>Generative AI with Blockchain</h1>")